In [1]:
import sys
sys.path.append('C:\\Users\\Shih-Yi\\Documents\\GitHub\\swac\\+swac_glm')

import os
import datajoint as dj
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# create virtual modules for swac
swac = dj.create_virtual_module('swac','swac')

[2023-07-21 10:38:18,616][INFO]: Connecting root@localhost:3306
[2023-07-21 10:38:18,683][INFO]: Connected root@localhost:3306


In [2]:
from pynwb import NWBHDF5IO, NWBFile
from pynwb.file import Subject
from pynwb.ophys import ImageSegmentation, OpticalChannel
from pynwb.base import Images
from pynwb.image import GrayscaleImage, ImageSeries

from datetime import datetime, timedelta
from dateutil import tz
import h5py


In [3]:
from ndx_harvey_swac import LabMetaDataMouse

In [4]:
sugery_note = {
    3:'cranial window creation date:2017-04-03, 3.5 mm diameter, left posterior cortex; AAVretro injection date:2017-04-03; GCaMP6s injection date:2017-04-03; performed by Shih-Yi Tseng and Selmaan N. Chettih',
    4:'cranial window creation date:2017-05-03, 3.5 mm diameter, left posterior cortex; AAVretro injection date:2017-05-03; GCaMP6s injection date:2017-05-03; performed by Shih-Yi Tseng and Selmaan N. Chettih',
    5:'cranial window creation date:2017-05-29, 3.5 mm diameter, left posterior cortex; AAVretro injection date:2017-05-29; GCaMP6s injection date:2017-05-29; performed by Shih-Yi Tseng and Selmaan N. Chettih',
    6:'cranial window creation date:2017-06-05, 3.5 mm diameter, left posterior cortex; AAVretro injection date:2017-06-05; GCaMP6s injection date:2017-06-12; performed by Shih-Yi Tseng and Selmaan N. Chettih',
    7:'cranial window creation date:2017-05-12, 3.5 mm diameter, left posterior cortex; AAVretro injection date:2017-05-12; GCaMP6s injection date:2017-07-05; performed by Shih-Yi Tseng and Selmaan N. Chettih',
    8:'cranial window creation date:2017-05-17, 3.5 mm diameter, left posterior cortex; AAVretro injection date:2017-05-17; GCaMP6s injection date:2017-07-12; performed by Shih-Yi Tseng and Selmaan N. Chettih',
    9:'cranial window creation date:2017-05-25, 3.5 mm diameter, left posterior cortex; AAVretro injection date:2017-05-25; GCaMP6s injection date:2017-07-26; performed by Shih-Yi Tseng and Selmaan N. Chettih',
    10:'cranial window creation date:2017-05-26, 3.5 mm diameter, left posterior cortex; AAVretro injection date:2017-05-26; GCaMP6s injection date:2017-09-11; performed by Shih-Yi Tseng and Selmaan N. Chettih'
}

In [5]:
virus_note = {
    3:'''AAV2/1-synapsin-GCaMP6s-WPRE-SV40 in left V1 x3 sites, PM x1, AM x0, MM x1, RSC x1, visA x1, 1/10 dilution, 140nl per site in L23 and 140nl per site in L5; AAV2retro-Syn-mTagBFP2 in left anterior ACC/M2 x3, undiluted, 300 nl per site, coordinate (mm from bregma): (A 0.95, L 0.6, D 1.0), (A 0.95, L 0.6, D 0.3), (A 0.95, L 0.8, D 0.3); AAV2retro-Syn-mScarlet in left striatum x3, 1/20 dilution, 300 nl per site, coordinate (mm from bregma): (A 0.95, L 1.2, D 2.1), (A 1.0, L 1.75, D 2.1), (P 0.15, L 2.15, D 2.1); mScarlet expression did not come up when examining it under the 2P microscope (due to the higher dilution)''',
    4:'''AAV2/1-synapsin-GCaMP6s-WPRE-SV40 in left V1 x3 sites, PM x1, AM x0, MM x1, RSC x0, visA x2, 1/10 dilution, 100nl per site in L23 and 140nl per site in L5; AAV2retro-Syn-mTagBFP2 in left anterior ACC/M2 x3, undiluted, 300 nl per site, coordinate (mm from bregma): (A 0.9, L 0.58, D 0.3), (A 0.9, L 0.58, D 1.0), (A 0.9, L 0.8, D 0.4); AAV2retro-Syn-mScarlet in left striatum x3, 1/5 dilution, 300 nl per site for the anterior sites and 600 nl for the posterior site, coordinate (mm from bregma): (A 1.0, L 1.2, D 2.1), (A 1.0, L 1.5, D 2.1), (P 0.2, L 1.75, D 2.1)''',
    5:'''AAV2/1-synapsin-GCaMP6s-WPRE-SV40 in left V1 x3 sites, PM x1, AM x1, MM x1, RSC x0, visA x2, 1/5 dilution, 100nl per site in L23 and 140nl per site in L5; AAV2retro-Syn-mTagBFP2 in left anterior ACC/M2 x3, undiluted, 300 nl per site, coordinate (mm from bregma): (A 0.96, L 0.4, D 0.3), (A 0.96, L 0.4, D 1.0), (A 0.96, L 0.8, D 0.4); AAV2retro-Syn-mScarlet in left striatum x3, 1/5 dilution, 300 nl per site for the anterior sites and 600 nl for the posterior site, coordinate (mm from bregma): (A 0.96, L 1.2, D 2.1), (A 0.96, L 1.5, D 2.1), (P 0.24, L 1.8, D 2.1)''',
    6:'''AAV2/1-synapsin-GCaMP6s-WPRE-SV40 in left V1 x3 sites, PM x1, AM x2, MM x1, RSC x1, visA x2, 1/10 dilution, 100nl per site in L23 and 140nl per site in L5; AAV2retro-Syn-mTagBFP2 in left posterior ACC/M2 x4, undiluted, 300 nl per site, coordinate (mm from bregma): (0, L 0.3, D 0.4), (0, L 0.3, D 0.8), (0, L 0.7, D 0.3), (0, L 0.7, D 0.75); AAV2retro-Syn-mScarlet in left ORBvl x1, ORBl x1, 1/5 dilution, 500 nl per site, coordinate (mm from bregma): (A 2.45, L 0.75, D 1.85), (A 2.45, L 1.26, D 1.85)''',
    7:'''AAV2/1-synapsin-GCaMP6s-WPRE-SV40 in left V1 x2 sites, PM x1, AM x1, MM x1, RSC x1, visA x2, 1/10 dilution, 70nl per site in L23 and 100nl per site in L5; AAV2retro-Syn-mTagBFP2 in left anterior ACC/M2 x3, undiluted, 300 nl per site, coordinate (mm from bregma): (A 0.7, L 0.5, D 0.3), (A 0.7, L 0.5, D 1.0), (A 1.0, L 0.8, D 0.4); AAV2retro-Syn-mScarlet in left striatum x3, 1/5 dilution, 300 nl per site for the anterior sites and 600 nl for the posterior site, coordinate (mm from bregma): (A 0.8, L 1.2, D 2.1), (A 1.15, L 1.5, D 2.1), (P 0.2, L 1.8, D 2.1)''',
    8:'''AAV2/1-synapsin-GCaMP6s-WPRE-SV40 in left V1 x2 sites, PM x1, AM x1, MM x1, RSC x1, visA x2, 1/10 dilution, 70nl per site in L23 and 100nl per site in L5; AAV2retro-Syn-mTagBFP2 in left anterior ACC/M2 x3, undiluted, 300 nl per site, coordinate (mm from bregma): (A 1.1, L 0.55, D 0.3), (A 1.1, L 0.55, D 1.0), (A 0.97, L 0.8, D 0.4); AAV2retro-Syn-mScarlet in left striatum x3, 1/5 dilution, 300 nl per site for the anterior sites and 600 nl for the posterior site, coordinate (mm from bregma): (A 0.98, L 1.2, D 2.1), (A 0.85, L 1.5, D 2.1), (P 0.152, L 1.8, D 2.1)''',
    9:'''AAV2/1-synapsin-GCaMP6s-WPRE-SV40 in left V1 x2 sites, PM x1, AM x1, MM x1, RSC x1, visA x2, 1/10 dilution, 70nl per site in L23 and 100nl per site in L5; AAV2retro-Syn-mTagBFP2 in left posterior ACC/M2 x4, undiluted, 300 nl per site, coordinate (mm from bregma): (0, L 0.35, D 0.4), (0, L 0.35, D 0.8), (0, L 0.7, D 0.3), (0, L 0.7, D 0.8); AAV2retro-Syn-mScarlet in left ORBvl x1, ORBl x1, 1/5 dilution, 500 nl per site, coordinate (mm from bregma): (A 2.65, L 0.85, D 1.8), (A 2.6, L 1.35, D 1.75)''',
    10:'''AAV2/1-synapsin-GCaMP6s-WPRE-SV40 in left V1 x2 sites, PM x1, AM x1, MM x1, RSC x1, visA x2, 1/10 dilution, 70nl per site in L23 and 100nl per site in L5; AAV2retro-Syn-mTagBFP2 in left posterior ACC/M2 x4, undiluted, 300 nl per site, coordinate (mm from bregma): (A 0.25, L 0.3, D 0.4), (A 0.25, L 0.3, D 0.85), (A 0.1, L 0.7, D 0.3), (A 0.1, L 0.7, D 0.8); AAV2retro-Syn-mScarlet in left  ORBvl x1, ORBl x1, 1/5 dilution, 500 nl per site, coordinate (mm from bregma): (A 2.45, L 0.75, D 1.8), (A 2.45, L 1.25, D 1.85)'''
}

In [6]:
retro_site = {
    3:{'mTagBFP2':'anterior_ACC_M2','mScarlet':'striatum'},
    4:{'mTagBFP2':'anterior_ACC_M2','mScarlet':'striatum'},
    5:{'mTagBFP2':'anterior_ACC_M2','mScarlet':'striatum'},
    6:{'mTagBFP2':'posterior_ACC_M2','mScarlet':'OFC'},
    7:{'mTagBFP2':'anterior_ACC_M2','mScarlet':'striatum'},
    8:{'mTagBFP2':'anterior_ACC_M2','mScarlet':'striatum'},
    9:{'mTagBFP2':'posterior_ACC_M2','mScarlet':'OFC'},
    10:{'mTagBFP2':'posterior_ACC_M2','mScarlet':'OFC'}
}

In [7]:
# specify mouse
key = {'mouse_id':3}
mouse_id = key['mouse_id']

In [8]:
# fetch widefiled data
wf_date, wf_param, wf_vessel, retinotopy, fft_power, field_sign, focal_depth = (swac.Widefield & key & 'best="T"').fetch1(
    'wf_date','wf_param','vessel', 'retinotopy', 'fft_power', 'field_sign','depth')


In [9]:
# create NWB file
nwbfile = NWBFile(
    session_description = 'Widefield retinotopy and 2p window vessel and retrograde label overview',
    identifier = f'Mouse_{mouse_id}_widefield_retinotopy_and_window_vessel',
    session_start_time = datetime.strptime(str(wf_date), '%Y-%m-%d'),
    session_id = f'Mouse_{mouse_id}_widefield_retinotopy_and_window_vessel', 
    keywords = ['cortex', 'retinotopy','widefield imaging'],
    surgery = sugery_note[mouse_id],
    virus = virus_note[mouse_id],
    experimenter = 'Tseng, Shih-Yi',   
    experiment_description = 'Widefield retinotopy and 2p window vessel and retrograde label overview',
    lab = 'Harvey Lab',                                       
    institution = 'Harvard Medical School',               
    related_publications = 'doi:10.1016/j.neuron.2022.05.012'
)

C:\Users\Shih-Yi\Anaconda3\envs\dandi_new\Lib\site-packages\pynwb\file.py:470: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['session_start_time'] = _add_missing_timezone(session_start_time)


In [10]:
# fetch mouse info
mouse_name, doa, training_start = (swac.Mouse & key).fetch1('name', 'doa', 'training_start')

# add subject information to NWB file
nwbfile.subject = Subject(
    subject_id = str(mouse_id),
    date_of_birth = datetime.combine(doa-timedelta(days=56), datetime.min.time()), # date of arrival minus 8 weeks
    description = f'Mouse {mouse_id} {mouse_name}, date of arrival {doa} (~8 weeks old; date of birth is approximated), training started on {training_start}',
    species = 'Mus musculus', 
    sex = 'M',
    strain = 'C57BL/6J',
    genotype = 'wildtype'
)

C:\Users\Shih-Yi\Anaconda3\envs\dandi_new\Lib\site-packages\pynwb\file.py:145: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['date_of_birth'] = _add_missing_timezone(date_of_birth)


In [11]:
# create processing modules for retinotopy
retino_module = nwbfile.create_processing_module(
    name = 'retinotopy', 
    description = 'widefield and cranial window vessel image'
)

# put WF imaging in GrayscaleImage container
field_sign_nwb = GrayscaleImage(
    name = 'field_sign_map',
    data = field_sign,
    description = 'field sign map collected with widefield imaging'
)
wf_vessel_nwb = GrayscaleImage(
    name = 'widefield_vessel_image',
    data = wf_vessel,
    description = 'image of vessel pattern at brain surface for the whole window collected with widefield imaging'
)
axis_1_phase = GrayscaleImage(
    name = 'axis_1_phase_map_altitude',
    data = retinotopy[:,:,0], 
    description = 'axis 1 (altitude) phase map in degree collected with widefield imaging'
)
axis_2_phase = GrayscaleImage(
    name = 'axis_2_phase_map_azimuth',
    data = retinotopy[:,:,1], 
    description = 'axis 2 (azimuth) phase map in degree collected with widefield imaging'
)
power_map = GrayscaleImage(
    name = 'power_map_avg',
    data = fft_power, 
    description = 'average FFT power map collected with widefield imaging'
)

# put images in an Images container
widefield = Images(
    name = 'widefield_imaging',
    images = [field_sign_nwb, wf_vessel_nwb, axis_1_phase, axis_2_phase, power_map],
    description = 'results of widefield imaging in this mouse. Use the Registration__wf_to_rs_mm_transformation_matrix in lab_meta_data to algin the WF images to the resize/processed images.',
)
# add widefield to the registration_module
retino_module.add(widefield)

#    from pynwb.retinotopy import RetinotopyImage, RetinotopyMap, AxisMap, ImagingRetinotopy, FocalDepthImage
#    
#     # put widefield data into corresponding containers
#     field_sign_nwb = RetinotopyMap(
#         name='field_sign_map', 
#         data=field_sign, 
#         field_of_view=[0.0035,0.0035], 
#         dimension=[512,512]
#         )
#     wf_vessel_nwb = RetinotopyImage(
#         name='widefield_vessel_image', 
#         data=wf_vessel, 
#         bits_per_pixel=16,
#         dimension=[512,512], 
#         format='raw', 
#         field_of_view=[0.0035,0.0035]
#     )
#     axis_1_phase = AxisMap(
#         name='axis_1_phase_map_altitude', 
#         data=retinotopy[:,:,0], 
#         field_of_view=[0.0035,0.0035],  
#         unit='degree', 
#         dimension=[512,512]
#     )
#     axis_2_phase = AxisMap(
#         name='axis_1_phase_map_azimuth', 
#         data=retinotopy[:,:,1], 
#         field_of_view=[0.0035,0.0035],  
#         unit='degree', 
#         dimension=[512,512]
#     )
#     power_map = AxisMap(
#         name='power_map_avg', 
#         data=fft_power/fft_power.max(), 
#         field_of_view=[0.0035,0.0035],  
#         unit='degree', 
#         dimension=[512,512]
#     )
#     focal_depth_image = FocalDepthImage(
#         name='focal_depth_image', 
#         data=[], 
#         bits_per_pixel=16,
#         dimension=[512,512], 
#         format='raw', 
#         field_of_view=[0.0035,0.0035],  
#         focal_depth=focal_depth/1000000
#     )
    
#     retinotopy_nwb = ImagingRetinotopy(
#         sign_map=field_sign_nwb, 
#         axis_1_phase_map=axis_1_phase, 
#         axis_1_power_map=axis_2_phase, 
#         axis_2_phase_map=power_map, 
#         axis_2_power_map=power_map, 
#         axis_descriptions=['altitude', 'azimuth'] , 
#         focal_depth_image=focal_depth_image, 
#         vasculature_image=wf_vessel_nwb, 
#         name='ImagingRetinotopy'
#     )
    
#     # add window vessel image to the registration_module
#     retino_module.add(retinotopy_nwb)
#     retino_module.add(field_sign_nwb)
#     retino_module.add(wf_vessel_nwb)
#     retino_module.add(axis_1_phase)
#     retino_module.add(axis_2_phase)
#     retino_module.add(power_map)

widefield_imaging pynwb.base.Images at 0x1230461281872
Fields:
  description: results of widefield imaging in this mouse. Use the Registration__wf_to_rs_mm_transformation_matrix in lab_meta_data to algin the WF images to the resize/processed images.
  images: {
    axis_1_phase_map_altitude <class 'pynwb.image.GrayscaleImage'>,
    axis_2_phase_map_azimuth <class 'pynwb.image.GrayscaleImage'>,
    field_sign_map <class 'pynwb.image.GrayscaleImage'>,
    power_map_avg <class 'pynwb.image.GrayscaleImage'>,
    widefield_vessel_image <class 'pynwb.image.GrayscaleImage'>
  }

In [12]:
# create processing modules for 2p imaging 
ophys_module = nwbfile.create_processing_module(
    name = 'ophys', 
    description = 'cranial window vessel image under 2P microscope'
)

# create Device, OptionalChannel and ImagingPlane
device = nwbfile.create_device(
    name = 'two_photon_microscope', 
    description = 'Two-photon microscope',
    manufacturer = 'Harvey Lab custom-bulit two-photon microscope (Odin)'
)

# create OpticalChannel for mTagBFP2
optical_channel_mTagBFP2 = OpticalChannel(
    name = 'OpticalChannel_mTagBFP2', 
    description = 'optical channel for mTagBFP2', 
    emission_lambda = 450.
)

# create OpticalChannel for mScarlet
optical_channel_mScarlet = OpticalChannel(
    name = 'OpticalChannel_mScarlet', 
    description = 'optical channel for mScarlet', 
    emission_lambda = 625.
)

# compute mm per pixel
fov_ap_size = 675./1000
fov_ml_size = 750./1000
mm_per_pixel_ap = fov_ap_size/512
mm_per_pixel_ml = fov_ml_size/512

# create imaging plane for mTagBFP2
imaging_plane_mTagBFP2 = nwbfile.create_imaging_plane(
    name = 'ImagingPlane_mTagBFP2_850nm',
    optical_channel = optical_channel_mTagBFP2,
    imaging_rate = 30.,
    description = 'vessel pattern at brain surface and mTagBFP2 images for the whole window with stitching FOVs',
    device = device,
    excitation_lambda = 850.,
    indicator = 'mTagBFP2',
    location = 'posterior cortex',
    grid_spacing = [mm_per_pixel_ap, mm_per_pixel_ml],
    grid_spacing_unit = 'mm',
    origin_coords = [np.nan, np.nan],
    origin_coords_unit = 'mm',
    reference_frame = f'[p->a, l->m] in minimeters from bregma in Allen Institute Mouse Common Coordinate Framework (CCF); i.e. directionality of the imaging FOV: x-axis: posterior to anterior; y-axis: laterl (left) to medial (right); however, the FOV axes does not run parallelly to the CCF coordinate system. Use the Registration__rs_to_ccf_mm_transformation_matrix in lab_meta_data to transform the resize/processed image to CCF.'
)

# create imaging plane for mScarlet
if mouse_id != 3:
    imaging_plane_mScarlet = nwbfile.create_imaging_plane(
        name = 'ImagingPlane_mScarlet_800nm',
        optical_channel = optical_channel_mScarlet,
        imaging_rate = 30.,
        description = 'mScarlet images for the whole window with stitching FOVs',
        device = device,
        excitation_lambda = 800.,
        indicator = 'mScarlet',
        location = 'posterior cortex',
        grid_spacing = [mm_per_pixel_ap, mm_per_pixel_ml],
        grid_spacing_unit = 'mm',
        origin_coords = [np.nan, np.nan],
        origin_coords_unit = 'mm',
        reference_frame = f'[p->a, l->m] in minimeters from bregma in Allen Institute Mouse Common Coordinate Framework (CCF); i.e. directionality of the imaging FOV: x-axis: posterior to anterior; y-axis: laterl (left) to medial (right); however, the FOV axes does not run parallelly to the CCF coordinate system. Use the Registration__rs_to_ccf_mm_transformation_matrix in lab_meta_data to transform the resize/processed image to CCF.'
    )

In [13]:
# fetch whole window vessel image (raw)
vessel_2p = (swac.Retrostack2P & key).fetch1('vessel_2p')

# fetch whole window vessel image (processed and resized)
rs_vessel_resize, ap_mm, ml_mm, rs_pix_per_mm = (swac.RetroVesselProcessing & key).fetch1('rs_vessel_resize', 'ap_mm', 'ml_mm', 'rs_pix_per_mm')
rs_mm_per_pix = 1./rs_pix_per_mm

# fetch resized/rotated 2p vessel img, and the transformation matrix from widefield to resized/rotated 2p vessel
tform_wf_to_rs, rs_vessel_reg = (swac.FieldsignRegistration & key).fetch1('tform_wf_to_rs','rs_vessel_reg')

# fetch registration information to CCF
window_center_mm, rs_center_mm, rs_to_ccf_tform_mm = (swac.RegToAllenCCF & key).fetch1('window_center_mm', 'rs_center_mm', 'tform_mm')

# put 2p vessel raw image in an Images container
vessel_2p_raw = GrayscaleImage(
    name = 'raw_whole_window_vessel_img_2p',
    data = vessel_2p,
    description = 'raw image of vessel pattern at brain surface for the whole window with stitching FOVs under 2P microscope'
)

# put 2p vessel resized image in an Images container
vessel_2p_resize = GrayscaleImage(
    name = 'processed_whole_window_vessel_img_2p',
    data = rs_vessel_reg, #np.fliplr(np.rot90(rs_vessel_resize)),
    resolution = rs_pix_per_mm*10, # covnert to pixels/cm as required
    description = f'processed image of stitched vessel pattern (remove the overlaping regions, resize to {rs_pix_per_mm} pixels per mm, and rotate 90 degree + fliplr so now x axis is ~lateral to medial, y axis is ~posterior to anterior, same orientation as WF vessel). Use the Registration__rs_to_ccf_mm_transformation_matrix in lab_meta_data to transform the resize/processed image to CCF.'
)


# put image in an Images container
window_vessel_2p = Images(
    name = 'whole_window_vessel_img_2p',
    images = [vessel_2p_raw, vessel_2p_resize],
    description = 'raw and processed images of vessel pattern at brain surface for the whole window with stitching FOVs under 2P microscope',
)

# add window vessel image to the registration_module
ophys_module.add(window_vessel_2p)

whole_window_vessel_img_2p pynwb.base.Images at 0x1230465954512
Fields:
  description: raw and processed images of vessel pattern at brain surface for the whole window with stitching FOVs under 2P microscope
  images: {
    processed_whole_window_vessel_img_2p <class 'pynwb.image.GrayscaleImage'>,
    raw_whole_window_vessel_img_2p <class 'pynwb.image.GrayscaleImage'>
  }

In [14]:
# fetch whole window retrolabel images (raw)
bl2, bl3, bl5, rl2, rl3, rl5 = (swac.RetroImage & key).fetch1('bl2', 'bl3', 'bl5', 'rl2', 'rl3', 'rl5')

# fetch whole window retrolabel images (processed and resized)
bl2_resize, bl3_resize, bl5_resize, rl2_resize, rl3_resize, rl5_resize = (swac.RetroImgProcessing & key).fetch1('bl2_resize', 'bl3_resize', 'bl5_resize', 'rl2_resize', 'rl3_resize', 'rl5_resize')

# put raw retrolabel images in an Images container
bl2_raw = GrayscaleImage(
    name = 'raw_whole_window_mTagBFP2_img_L2',
    data = bl2,
    description = 'raw mTagBFP2 image in L2 for the whole window with stitching FOVs under 2P microscope'
)

bl3_raw = GrayscaleImage(
    name = 'raw_whole_window_mTagBFP2_img_L3',
    data = bl3,
    description = 'raw mTagBFP2 image in L3 for the whole window with stitching FOVs under 2P microscope'
)

bl5_raw = GrayscaleImage(
    name = 'raw_whole_window_mTagBFP2_img_L5',
    data = bl5,
    description = 'raw mTagBFP2 image in L5 for the whole window with stitching FOVs under 2P microscope'
)

if mouse_id != 3:
    rl2_raw = GrayscaleImage(
        name = 'raw_whole_window_mScarlet_img_L2',
        data = rl2,
        description = 'raw mScarlet image in L2 for the whole window with stitching FOVs under 2P microscope'
    )

    rl3_raw = GrayscaleImage(
        name = 'raw_whole_window_mScarlet_img_L3',
        data = rl3,
        description = 'raw mScarlet image in L3 for the whole window with stitching FOVs under 2P microscope'
    )

    rl5_raw = GrayscaleImage(
        name = 'raw_whole_window_mScarlet_img_L5',
        data = rl5,
        description = 'raw mScarlet image in L5 for the whole window with stitching FOVs under 2P microscope'
    )

# put resized and rotated retrolabel image in an Images container
bl2_resize = GrayscaleImage(
    name = 'processed_whole_window_mTagBFP2_img_L2',
    data = np.fliplr(np.rot90(bl2_resize)),
    resolution = rs_pix_per_mm*10, # covnert to pixels/cm as required
    description = f'processed image of stitched mTagBFP2 image in L2 (remove the overlaping regions, resize to {rs_pix_per_mm} pixels per mm, and rotate 90 degree + fliplr so now x axis is ~lateral to medial, y axis is ~posterior to anterior, same orientation as WF vessel).Use the Registration__rs_to_ccf_mm_transformation_matrix in lab_meta_data to transform the resize/processed image to CCF.'
)

bl3_resize = GrayscaleImage(
    name = 'processed_whole_window_mTagBFP2_img_L3',
    data = np.fliplr(np.rot90(bl3_resize)),
    resolution = rs_pix_per_mm*10, # covnert to pixels/cm as required
    description = f'processed image of stitched mTagBFP2 image in L3 (remove the overlaping regions, resize to {rs_pix_per_mm} pixels per mm, and rotate 90 degree + fliplr so now x axis is ~lateral to medial, y axis is ~posterior to anterior, same orientation as WF vessel).Use the Registration__rs_to_ccf_mm_transformation_matrix in lab_meta_data to transform the resize/processed image to CCF.'
)

bl5_resize = GrayscaleImage(
    name = 'processed_whole_window_mTagBFP2_img_L5',
    data = np.fliplr(np.rot90(bl5_resize)),
    resolution = rs_pix_per_mm*10, # covnert to pixels/cm as required
    description = f'processed image of stitched mTagBFP2 image in L5 (remove the overlaping regions, resize to {rs_pix_per_mm} pixels per mm, and rotate 90 degree + fliplr so now x axis is ~lateral to medial, y axis is ~posterior to anterior, same orientation as WF vessel).Use the Registration__rs_to_ccf_mm_transformation_matrix in lab_meta_data to transform the resize/processed image to CCF.'
)

if mouse_id != 3:
    rl2_resize = GrayscaleImage(
        name = 'processed_whole_window_mScarlet_img_L2',
        data = np.fliplr(np.rot90(rl2_resize)),
        resolution = rs_pix_per_mm*10, # covnert to pixels/cm as required
        description = f'processed image of stitched mScarlet image in L2 (remove the overlaping regions, resize to {rs_pix_per_mm} pixels per mm, and rotate 90 degree + fliplr so now x axis is ~lateral to medial, y axis is ~posterior to anterior, same orientation as WF vessel).Use the Registration__rs_to_ccf_mm_transformation_matrix in lab_meta_data to transform the resize/processed image to CCF.'
    )

    rl3_resize = GrayscaleImage(
        name = 'processed_whole_window_mScarlet_img_L3',
        data = np.fliplr(np.rot90(rl3_resize)),
        resolution = rs_pix_per_mm*10, # covnert to pixels/cm as required
        description = f'processed image of stitched mScarlet image in L3 (remove the overlaping regions, resize to {rs_pix_per_mm} pixels per mm, and rotate 90 degree + fliplr so now x axis is ~lateral to medial, y axis is ~posterior to anterior, same orientation as WF vessel).Use the Registration__rs_to_ccf_mm_transformation_matrix in lab_meta_data to transform the resize/processed image to CCF.'
    )

    rl5_resize = GrayscaleImage(
        name = 'processed_whole_window_mScarlet_img_L5',
        data = np.fliplr(np.rot90(rl5_resize)),
        resolution = rs_pix_per_mm*10, # covnert to pixels/cm as required
        description = f'processed image of stitched mScarlet image in L5 (remove the overlaping regions, resize to {rs_pix_per_mm} pixels per mm, and rotate 90 degree + fliplr so now x axis is ~lateral to medial, y axis is ~posterior to anterior, same orientation as WF vessel)'
    )


# put mTagBFP2 image in an Images container
window_mTagBFP2_2p = Images(
    name = 'whole_window_mTagBFP2_img_2p',
    images = [bl2_raw, bl3_raw, bl5_raw, bl2_resize, bl3_resize, bl5_resize],
    description = 'raw and processed mTagBFP2 images for the whole window with stitching FOVs under 2P microscope',
)
# add mTagBFP2 image to the registration_module
ophys_module.add(window_mTagBFP2_2p)

if mouse_id != 3:
# put mScarlet image in an Images container
    window_mScarlet_2p = Images(
        name = 'whole_window_mScarlet_img_2p',
        images = [rl2_raw, rl3_raw, rl5_raw, rl2_resize, rl3_resize, rl5_resize],
        description = 'raw and processed mScarlet images for the whole window with stitching FOVs under 2P microscope',
    )

    # add mScarlet image to the registration_module
    ophys_module.add(window_mScarlet_2p)

In [15]:
# add lab meta data
lab_meta_data = LabMetaDataMouse(
    Registration__window_center_ml_ccf_mm=window_center_mm.flatten()[0],
    Registration__window_center_ap_ccf_mm=window_center_mm.flatten()[1],
    Registration__vessel_2p_center_ml_ccf_mm=rs_center_mm.flatten()[0],
    Registration__vessel_2p_center_ap_ccf_mm=rs_center_mm.flatten()[1],
    Registration__rs_to_ccf_mm_transformation_matrix=rs_to_ccf_tform_mm,
    Registration__wf_to_rs_transformation_matrix=tform_wf_to_rs,
    AAVretroInjSite__mTagBFP2=retro_site[mouse_id]['mTagBFP2'],
    AAVretroInjSite__mScarlet=retro_site[mouse_id]['mScarlet']
)
    
nwbfile.add_lab_meta_data(lab_meta_data=lab_meta_data)    

harvey_lab_swac_metadata_mouse abc.LabMetaDataMouse at 0x1229613890384
Fields:
  AAVretroInjSite__mScarlet: OFC
  AAVretroInjSite__mTagBFP2: posterior_ACC_M2
  Registration__rs_to_ccf_mm_transformation_matrix: [[ 9.72427841e-03  8.89295743e-04 -3.66937526e+00]
 [ 8.68215883e-04 -9.96037916e-03 -1.22290643e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
  Registration__vessel_2p_center_ap_ccf_mm: -2.4373252341800686
  Registration__vessel_2p_center_ml_ccf_mm: -2.09067857418826
  Registration__wf_to_rs_transformation_matrix: [[   5.09051687    0.53987447 -359.69827574]
 [  -0.53987447    5.09051687 -304.79568428]
 [   0.            0.            1.        ]]
  Registration__window_center_ap_ccf_mm: -2.3581631449424245
  Registration__window_center_ml_ccf_mm: -1.9246658745816219

In [16]:
nwbfile

root pynwb.file.NWBFile at 0x1230464072528
Fields:
  devices: {
    two_photon_microscope <class 'pynwb.device.Device'>
  }
  experiment_description: Widefield retinotopy and 2p window vessel and retrograde label overview
  experimenter: ['Tseng, Shih-Yi']
  file_create_date: [datetime.datetime(2023, 7, 21, 10, 38, 21, 955419, tzinfo=tzlocal())]
  identifier: Mouse_10_widefield_retinotopy_and_window_vessel
  imaging_planes: {
    ImagingPlane_mScarlet_800nm <class 'pynwb.ophys.ImagingPlane'>,
    ImagingPlane_mTagBFP2_850nm <class 'pynwb.ophys.ImagingPlane'>
  }
  institution: Harvard Medical School
  keywords: ['cortex' 'retinotopy' 'widefield imaging']
  lab: Harvey Lab
  lab_meta_data: {
    harvey_lab_swac_metadata_mouse <class 'abc.LabMetaDataMouse'>
  }
  processing: {
    ophys <class 'pynwb.base.ProcessingModule'>,
    retinotopy <class 'pynwb.base.ProcessingModule'>
  }
  related_publications: ['doi:10.1016/j.neuron.2022.05.012']
  session_description: Widefield retinotopy and

In [17]:
# save nwb file
filePath = 'Z:\\HarveyLab\\Tier1\\Shih_Yi\\NWBfile\\'+ f'{mouse_id}\\mouse_{mouse_id}_widefield_and_2p_vessel.nwb'
with NWBHDF5IO(filePath, 'w') as io:
    io.write(nwbfile)